## Poetry Classification

In [ ]:
!pip install tkseem
!pip install tnkeeh

In [ ]:
!wget https://raw.githubusercontent.com/ARBML/tkseem/master/tasks/meter_classification/meters/data.txt
!wget https://raw.githubusercontent.com/ARBML/tkseem/master/tasks/meter_classification/meters/labels.txt

### Imports

In [3]:
import tensorflow as tf
import tkseem as tk
import tnkeeh as tn
import numpy as np
from tensorflow.keras.layers import GRU, Embedding, Dense, Input, Dropout, Bidirectional, BatchNormalization, Flatten, Reshape
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split

### Process data

In [5]:
tn.clean_data(file_path = 'meters/data.txt', save_path = 'meters/cleaned_data.txt', remove_diacritics=True, 
      execluded_chars=['!', '.', '?', '#'])
tn.split_classification_data('meters/cleaned_data.txt', 'meters/labels.txt')
train_data, test_data, train_lbls, test_lbls = tn.read_data(mode = 1)

Remove diacritics
Remove Tatweel
Saving to meters/cleaned_data.txt
Split data
Save to data
Read data  ['test_data.txt', 'test_lbls.txt', 'train_data.txt', 'train_lbls.txt']


In [6]:
max_length = max(len(data) for data in train_data)

### Tokenization

In [7]:
tokenizer = tk.CharacterTokenizer()
tokenizer.train('data/train_data.txt')

Training CharacterTokenizer ...


### Tokenize data

In [8]:
def preprocess(tokenizer, data, labels):
    X = tokenizer.encode_sentences(data)
    y = np.array([int(lbl) for lbl in labels])
    return X, y

In [9]:
# process training data
X_train, y_train = preprocess(tokenizer, train_data, train_lbls)

# process test data
X_test, y_test = preprocess(tokenizer, test_data, test_lbls)

### Model

In [11]:
model = Sequential()
model.add(Input((100,)))
model.add(Embedding(tokenizer.vocab_size, 256))
model.add(Bidirectional(GRU(units = 256, return_sequences=True)))
model.add(Bidirectional(GRU(units = 256, return_sequences=True)))
model.add(Bidirectional(GRU(units = 256)))
model.add(Dense(128, activation = 'relu'))
model.add(Dropout(0.3))
model.add(Dense(14, activation = 'softmax'))
model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
model.fit(X_train, y_train, validation_split = 0.1, epochs = 10, batch_size= 256, shuffle = True)

Epoch 1/10
133/133 [==============================] - 435s 3s/step - loss: 2.2590 - accuracy: 0.1899 - val_loss: 1.9318 - val_accuracy: 0.2881
Epoch 2/10
133/133 [==============================] - 399s 3s/step - loss: 1.7722 - accuracy: 0.3391 - val_loss: 1.6506 - val_accuracy: 0.3907
Epoch 3/10
133/133 [==============================] - 383s 3s/step - loss: 1.5203 - accuracy: 0.4517 - val_loss: 1.2544 - val_accuracy: 0.5645
Epoch 4/10
133/133 [==============================] - 385s 3s/step - loss: 1.0520 - accuracy: 0.6415 - val_loss: 0.7986 - val_accuracy: 0.7440
Epoch 5/10
133/133 [==============================] - 382s 3s/step - loss: 0.7140 - accuracy: 0.7734 - val_loss: 0.6061 - val_accuracy: 0.7997
Epoch 6/10
133/133 [==============================] - 381s 3s/step - loss: 0.5413 - accuracy: 0.8353 - val_loss: 0.5232 - val_accuracy: 0.8395
Epoch 7/10
133/133 [==============================] - 386s 3s/step - loss: 0.4214 - accuracy: 0.8757 - val_loss: 0.4718 - val_accuracy: 0.8589

### Test

In [11]:
label2name = ['السريع', 'الكامل', 'المتقارب', 'المتدارك', 'المنسرح', 'المديد', 
              'المجتث', 'الرمل', 'البسيط', 'الخفيف', 'الطويل', 'الوافر', 'الهزج', 'الرجز']

In [12]:
def classify(sentence):
    sequence = tokenizer.encode_sentences([sentence], out_length = max_length)
    pred = model.predict(sequence)[0]
    print(label2name[np.argmax(pred, 0).astype('int')], np.max(pred))

In [13]:
classify("ما تردون على هذا المحب # دائبا يشكو إليكم في الكتب")
classify("ولد الهدى فالكائنات ضياء # وفم الزمان تبسم وسناء")
classify(" لك يا منازل في القلوب منازل # أقفرت أنت وهن منك أواهل")
classify("ومن لم يمت بالسيف مات بغيره # تعددت الأسباب والموت واحد")
classify("أنا النبي لا كذب # أنا ابن عبد المطلب")
classify("هذه دراهم اقفرت # أم ربور محتها الدهور")
classify("هزجنا في بواديكم # فأجزلتم عطايانا")
classify("بحر سريع ماله ساحل # مستفعلن مستفعلن فاعلن")
classify("مَا مَضَى فَاتَ وَالْمُؤَمَّلُ غَيْبٌ # وَلَكَ السَّاعَةُ الَّتِيْ أَنْتَ فِيْهَا")
classify("يا ليلُ الصبّ متى غدهُ # أقيامُ الساعة موعدهُ")

الرمل 0.99949336
الكامل 0.9249198
الكامل 0.9218475
الطويل 0.9974923
المجتث 0.6107578
المديد 0.9062244
الهزج 0.9060462
السريع 0.832686
المتدارك 0.6292933
المتدارك 0.60360384
